#### Target variable : Risk decision on 6 levels
#### Feature: Subject (after sentence embedding) & hazard type (one-hot encoding), category, origin, notifying country (target encoding)

#### Model: CatBoost, XGBoost

In [3]:
import pandas as pd

df=pd.read_csv("/content/rasff_new2.csv")

In [4]:
# ----------------------
# Load and prepare data
# ----------------------
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

# ----------------------
# Time-based split (80/20)
# ----------------------
cutoff_index = int(len(df) * 0.8)
cutoff_date = df.iloc[cutoff_index]['date']

train_df = df[df['date'] <= cutoff_date]
test_df  = df[df['date'] > cutoff_date]

In [5]:
%pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 4.8 MB/s eta 0:00:00


In [7]:
from category_encoders import CatBoostEncoder
from sentence_transformers import SentenceTransformer

In [8]:
# ----------------------
# Sentence Transformer Embedding
# ----------------------
model = SentenceTransformer('all-MiniLM-L6-v2')

train_embeddings = model.encode(train_df['subject'].tolist(), show_progress_bar=True)
test_embeddings  = model.encode(test_df['subject'].tolist(), show_progress_bar=True)

# Add embedding vectors back into dataframe (384 dims)
emb_cols = [f"sub_emb_{i}" for i in range(train_embeddings.shape[1])]

train_df[emb_cols] = train_embeddings
test_df[emb_cols]  = test_embeddings

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/685 [00:00<?, ?it/s]

Batches:   0%|          | 0/172 [00:00<?, ?it/s]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/tmp/ipython-input-3329622934.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[emb_cols] = train_embeddings
/tmp/ipython-input-3329622934.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[emb_cols] = train_embeddings
/tmp/ipython-input-3329622934.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [9]:
df.head()

,reference,category,type,subject,date,notifying_country,classification,risk_decision,distribution,forAttention,forFollowUp,operator,origin,hazards,year,month,Hazard_Type,Regulatory_Issue_Flag
27396,2020.0001,fruits and vegetables,food,Aflatoxins in dried figs from Turkey,2020-01-02 09:24:48,France,border rejection notification,serious,NaN,NaN,NaN,Türkiye,Türkiye,Aflatoxins B1,2020,1,Mycotoxins,0
27395,2020.0004,fruits and vegetables,food,aflatoxins in dried figs from Turkey,2020-01-02 11:06:54,Italy,border rejection notification,serious,NaN,NaN,NaN,"Italy,Türkiye",Türkiye,aflatoxin total,2020,1,Mycotoxins,0
27394,2020.0007,fruits and vegetables,food,fosthiazate in fresh peppers from Turkey,2020-01-02 12:35:16,Bulgaria,border rejection notification,serious,NaN,NaN,NaN,"Germany,Türkiye",Türkiye,fosthiazate,2020,1,Chemical/Contaminants,0
27393,2020.0008,meat and meat products (other than poultry),food,Salmonella spp. (in 1 out of 5 samples) in Hot...,2020-01-02 12:35:36,Poland,alert notification,serious,Estonia,Poland,NaN,"Estonia,Poland",Poland,Salmonella enteritidis,2020,1,Microbiological,0
27392,2020.0010,food contact materials,food contact material,migration of formaldehyde form bamboo mug from...,2020-01-02 13:42:42,Poland,alert notification,serious,"Belarus,Lithuania,Poland,Slovakia",NaN,"Lithuania,Slovakia","China,Poland",China,formaldehyde migration,2020,1,Migration from Packaging,0


In [10]:
# ----------------------
# Target Encoding
# ----------------------
target_cols = ['origin', 'notifying_country', 'category']

encoder = CatBoostEncoder(cols=target_cols)
encoder.fit(train_df[target_cols], train_df['risk_decision'])

train_df[target_cols] = encoder.transform(train_df[target_cols])
test_df[target_cols]  = encoder.transform(test_df[target_cols])

/tmp/ipython-input-113083986.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[target_cols] = encoder.transform(train_df[target_cols])
/tmp/ipython-input-113083986.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[target_cols]  = encoder.transform(test_df[target_cols])


In [11]:
# ----------------------
# One-hot encoding hazard_type
# ----------------------
train_df = pd.get_dummies(train_df, columns=['Hazard_Type'])
test_df  = pd.get_dummies(test_df, columns=['Hazard_Type'])

# Match test columns to train
test_df = test_df.reindex(columns=train_df.columns, fill_value=0)

In [12]:
train_df.head()

,reference,category,type,subject,date,notifying_country,classification,risk_decision,distribution,forAttention,...,sub_emb_383,Hazard_Type_Allergen,Hazard_Type_Chemical/Contaminants,Hazard_Type_Fraud/Adulteration,Hazard_Type_Microbiological,Hazard_Type_Migration from Packaging,Hazard_Type_Mycotoxins,Hazard_Type_Novel/Unauthorised Ingredient,Hazard_Type_Others,Hazard_Type_Physical Contaminants
27396,2020.0001,3.690679,food,Aflatoxins in dried figs from Turkey,2020-01-02 09:24:48,3.484561,border rejection notification,serious,NaN,NaN,...,0.015402,False,False,False,False,False,True,False,False,False
27395,2020.0004,3.690679,food,aflatoxins in dried figs from Turkey,2020-01-02 11:06:54,3.363856,border rejection notification,serious,NaN,NaN,...,0.015402,False,False,False,False,False,True,False,False,False
27394,2020.0007,3.690679,food,fosthiazate in fresh peppers from Turkey,2020-01-02 12:35:16,4.034471,border rejection notification,serious,NaN,NaN,...,0.010728,False,True,False,False,False,False,False,False,False
27393,2020.0008,3.558318,food,Salmonella spp. (in 1 out of 5 samples) in Hot...,2020-01-02 12:35:36,3.076378,alert notification,serious,Estonia,Poland,...,0.035854,False,False,False,True,False,False,False,False,False
27392,2020.0010,2.751504,food contact material,migration of formaldehyde form bamboo mug from...,2020-01-02 13:42:42,3.076378,alert notification,serious,"Belarus,Lithuania,Poland,Slovakia",NaN,...,0.015101,False,False,False,False,True,False,False,False,False


In [13]:
# Save training and test datasets
train_df.to_pickle('rasff_train_processed.pkl')
test_df.to_pickle('rasff_test_processed.pkl')

In [14]:
import pandas as pd

train_df = pd.read_pickle('rasff_train_processed.pkl')
test_df  = pd.read_pickle('rasff_test_processed.pkl')

In [15]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.4 MB/s eta 0:00:00


In [22]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report, accuracy_score
# Compute weights inversely proportional to frequency
from sklearn.utils.class_weight import compute_class_weight

# -------------------------
# 1. Identify features
# -------------------------
# Assuming your train_df and test_df already have embeddings + encoded features
target_col = 'risk_decision'

# Exclude target and non-feature columns
non_features = ['reference', 'date', 'subject', 'distribution', 'forAttention', 'forFollowUp', 'operator', 'hazards', 'classification', target_col]
feature_cols = [col for col in train_df.columns if col not in non_features]

# -------------------------
# 2. Identify categorical features for CatBoost
# -------------------------
# Only include columns that are still strings/categorical
cat_features = ['type']  # adjust if you one-hot encoded hazard_type

# -------------------------
# 3. Create CatBoost Pools
# -------------------------
train_pool = Pool(
    data=train_df[feature_cols],
    label=train_df[target_col],
    cat_features=cat_features
)

test_pool = Pool(
    data=test_df[feature_cols],
    label=test_df[target_col],
    cat_features=cat_features
)

# -------------------------
# 4. Initialize and train CatBoost
# -------------------------

model_cb = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.03,
    depth=6,
    loss_function='MultiClass',
    eval_metric='Accuracy',
    task_type='GPU',
    early_stopping_rounds=100,
    random_seed=42,
    verbose=200
)

model_cb.fit(train_pool, eval_set=test_pool)

# -------------------------
# 5. Evaluate model
# -------------------------
preds = model_cb.predict(test_pool)
preds = preds.flatten()  # flatten if shape is (n,1)

accuracy = accuracy_score(test_df[target_col], preds)
print(f"Test Accuracy: {accuracy:.4f}\n")

print("Classification Report:")
print(classification_report(test_df[target_col], preds))

0:	learn: 0.6272926	test: 0.5263734	best: 0.5263734 (0)	total: 60.6ms	remaining: 2m 1s
200:	learn: 0.7138425	test: 0.5663442	best: 0.5668918 (195)	total: 5.27s	remaining: 47.2s
400:	learn: 0.7533534	test: 0.5796678	best: 0.5798503 (394)	total: 12.2s	remaining: 48.7s
600:	learn: 0.7799526	test: 0.5869684	best: 0.5873335 (579)	total: 16.4s	remaining: 38.2s
800:	learn: 0.7996168	test: 0.5898887	best: 0.5908012 (745)	total: 22.3s	remaining: 33.3s
bestTest = 0.5931739368
bestIteration = 884
Shrink model to first 885 iterations.
Test Accuracy: 0.5932

Classification Report:
                     precision    recall  f1-score   support

            no risk       0.21      0.05      0.08        65
        not serious       0.56      0.45      0.50       739
     potential risk       0.54      0.42      0.47      1078
potentially serious       0.52      0.12      0.19      1045
            serious       0.62      0.92      0.74      2552

           accuracy                           0.59      5

In [23]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report, accuracy_score
# Compute weights inversely proportional to frequency
from sklearn.utils.class_weight import compute_class_weight

# -------------------------
# 1. Identify features
# -------------------------
# Assuming your train_df and test_df already have embeddings + encoded features
target_col = 'risk_decision'

# Exclude target and non-feature columns
non_features = ['reference', 'date', 'subject', 'distribution', 'forAttention', 'forFollowUp', 'operator', 'hazards', 'classification', target_col]
feature_cols = [col for col in train_df.columns if col not in non_features]

# -------------------------
# 2. Identify categorical features for CatBoost
# -------------------------
# Only include columns that are still strings/categorical
cat_features = ['type']  # adjust if you one-hot encoded hazard_type

# -------------------------
# 3. Create CatBoost Pools
# -------------------------
train_pool = Pool(
    data=train_df[feature_cols],
    label=train_df[target_col],
    cat_features=cat_features
)

test_pool = Pool(
    data=test_df[feature_cols],
    label=test_df[target_col],
    cat_features=cat_features
)

# -------------------------
# 4. Initialize and train CatBoost
# -------------------------

model_cb = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.05,
    depth=8,
    loss_function='MultiClass',
    eval_metric='Accuracy',
    task_type='GPU',
    early_stopping_rounds=100,
    random_seed=42,
    verbose=200
)

model_cb.fit(train_pool, eval_set=test_pool)

# -------------------------
# 5. Evaluate model
# -------------------------
preds = model_cb.predict(test_pool)
preds = preds.flatten()  # flatten if shape is (n,1)

accuracy = accuracy_score(test_df[target_col], preds)
print(f"Test Accuracy: {accuracy:.4f}\n")

print("Classification Report:")
print(classification_report(test_df[target_col], preds))

0:	learn: 0.6495574	test: 0.5287461	best: 0.5287461 (0)	total: 130ms	remaining: 4m 19s
200:	learn: 0.7920431	test: 0.5876985	best: 0.5889761 (198)	total: 13.2s	remaining: 1m 58s
400:	learn: 0.8597500	test: 0.5944515	best: 0.5955466 (394)	total: 25.8s	remaining: 1m 42s
600:	learn: 0.9008577	test: 0.5993794	best: 0.6015696 (588)	total: 38s	remaining: 1m 28s
bestTest = 0.6021171747
bestIteration = 659
Shrink model to first 660 iterations.
Test Accuracy: 0.6021

Classification Report:
                     precision    recall  f1-score   support

            no risk       0.12      0.06      0.08        65
        not serious       0.57      0.44      0.50       739
     potential risk       0.55      0.44      0.49      1078
potentially serious       0.48      0.14      0.22      1045
            serious       0.63      0.92      0.75      2552

           accuracy                           0.60      5479
          macro avg       0.47      0.40      0.41      5479
       weighted avg     

In [46]:
print(train_df['risk_decision'].unique())

['serious' 'undecided' 'not serious' 'no risk' 'potentially serious'
 'potential risk']


In [49]:
import string

def clean_label(s):
    if pd.isna(s):
        return s
    # Lowercase
    s = s.lower()
    # Remove punctuation
    s = s.translate(str.maketrans('', '', string.punctuation))
    # Strip spaces
    s = s.strip()
    return s

train_df['risk_decision_clean'] = train_df['risk_decision'].apply(clean_label)
test_df['risk_decision_clean']  = test_df['risk_decision'].apply(clean_label)

In [50]:
risk_mapping = {
    'no risk': 0,
    'not serious': 1,
    'undecided': 2,
    'potential risk': 3,
    'potentially serious': 4,
    'serious': 5
}

train_df['risk_decision_num'] = train_df['risk_decision_clean'].map(risk_mapping)
test_df['risk_decision_num']  = test_df['risk_decision_clean'].map(risk_mapping)

In [51]:
print(train_df['risk_decision_num'].isna().sum())  # Should be 0
print(test_df['risk_decision_num'].isna().sum())   # Should be 0

0
0


In [65]:
# Identify features
# train_df and test_df already have embeddings + encoded features
# Define target column (After mapping)
target_col = 'risk_decision_num'

# Exclude target and non-feature columns
non_features = ['reference', 'type', 'date', 'month', 'year', 'subject', 'distribution', 'forAttention', 'forFollowUp', 'operator', 'hazards', 'classification','risk_decision','risk_decision_clean', target_col]
feature_cols = [col for col in train_df.columns if col not in non_features]


In [55]:
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score




# Prepare data
X_train = train_df[feature_cols].values
y_train = train_df[target_col].values
X_test  = test_df[feature_cols].values
y_test  = test_df[target_col].values

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest  = xgb.DMatrix(X_test, label=y_test)

# Define multi-class parameters
num_classes = len(train_df[target_col].unique())
params = {
    'objective': 'multi:softmax',  # or 'multi:softprob' for probabilities
    'num_class': num_classes,
    'eval_metric': 'mlogloss',
    'eta': 0.03,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

# Train
bst = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    evals=[(dtrain, 'train'), (dtest, 'test')],
    early_stopping_rounds=50,
    verbose_eval=50
)

# Predict
y_pred = bst.predict(dtest)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

[0]	train-mlogloss:1.58631	test-mlogloss:1.65342
[50]	train-mlogloss:0.96446	test-mlogloss:1.35869
[100]	train-mlogloss:0.76033	test-mlogloss:1.32194
[150]	train-mlogloss:0.64306	test-mlogloss:1.31349
[200]	train-mlogloss:0.56149	test-mlogloss:1.30830
[250]	train-mlogloss:0.49707	test-mlogloss:1.30649
[300]	train-mlogloss:0.44602	test-mlogloss:1.30702
[322]	train-mlogloss:0.42591	test-mlogloss:1.30831
Accuracy: 0.5411571454645008
              precision    recall  f1-score   support

         0.0       0.07      0.03      0.04        65
         1.0       0.50      0.51      0.50       739
         2.0       0.00      0.00      0.00         0
         3.0       0.70      0.10      0.18      1078
         4.0       0.59      0.04      0.07      1045
         5.0       0.59      0.96      0.73      2552

    accuracy                           0.54      5479
   macro avg       0.41      0.27      0.25      5479
weighted avg       0.59      0.54      0.46      5479



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [66]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# ---------------------------
# Prepare data
# ---------------------------
X_train = train_df[feature_cols].values
y_train = train_df[target_col].values
X_test  = test_df[feature_cols].values
y_test  = test_df[target_col].values

# Handle class imbalance via weights
classes = np.unique(y_train)
weights = np.bincount(y_train)
sample_weights = np.array([1.0 / weights[y] for y in y_train])

dtrain = xgb.DMatrix(X_train, label=y_train, weight=sample_weights)
dtest  = xgb.DMatrix(X_test, label=y_test)

num_classes = len(classes)
params = {
    'objective': 'multi:softmax',
    'num_class': num_classes,
    'learning_rate': 0.03,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'eval_metric': 'mlogloss'
}

# Train with early stopping
evals = [(dtrain, 'train'), (dtest, 'test')]
bst = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    evals=evals,
    early_stopping_rounds=50,
    verbose_eval=50
)

# Predict
y_pred = bst.predict(dtest)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

[0]	train-mlogloss:1.79176	test-mlogloss:1.79178
[50]	train-mlogloss:1.79176	test-mlogloss:1.79147
[91]	train-mlogloss:1.79176	test-mlogloss:1.79150
Accuracy: 0.4657784267202044
              precision    recall  f1-score   support

           0      0.000     0.000     0.000        65
           1      0.000     0.000     0.000       739
           3      0.000     0.000     0.000      1078
           4      0.000     0.000     0.000      1045
           5      0.466     1.000     0.636      2552

    accuracy                          0.466      5479
   macro avg      0.093     0.200     0.127      5479
weighted avg      0.217     0.466     0.296      5479



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [62]:
import xgboost as xgb
print(xgb.__version__)

3.1.2


In [61]:
pip install --upgrade xgboost

#### Target variable : Risk decision on 3 levels
#### Feature: Subject (after sentence embedding) & hazard type (one-hot encoding), category, origin, notifying country (target encoding)

#### Model: CatBoost, XGBoost, LightGBM

In [67]:
# ----------------------------
# Recode risk_decision into 3 classes
# ----------------------------
def recode_risk(risk):
    if risk in ['no risk', 'not serious']:
        return 0   # Low risk
    elif risk in ['potential risk', 'undecided', 'potentially serious']:
        return 1   # Medium / potential risk
    elif risk == 'serious':
        return 2   # High risk
    else:
        return -1  # Safety net for unexpected values

# Apply to both train and test
train_df['risk_decision_3class'] = train_df['risk_decision'].apply(recode_risk)
test_df['risk_decision_3class']  = test_df['risk_decision'].apply(recode_risk)

# Optional: check distribution
print(train_df['risk_decision_3class'].value_counts())
print(test_df['risk_decision_3class'].value_counts())

# Update target variable
target_col = 'risk_decision_3class'

risk_decision_3class
2    12204
1     5967
0     3747
Name: count, dtype: int64
risk_decision_3class
2    2552
1    2123
0     804
Name: count, dtype: int64


In [75]:
# Identify features
# train_df and test_df already have embeddings + encoded features
# Define target column (After mapping)
target_col = 'risk_decision_3class'

# Exclude target and non-feature columns
non_features = ['reference', 'type', 'date', 'month', 'year', 'subject', 'distribution', 'forAttention', 'forFollowUp', 'operator', 'hazards', 'classification','risk_decision','risk_decision_clean', 'risk_decision_num', target_col]
feature_cols = [col for col in train_df.columns if col not in non_features]

In [69]:
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score




# Prepare data
X_train = train_df[feature_cols].values
y_train = train_df[target_col].values
X_test  = test_df[feature_cols].values
y_test  = test_df[target_col].values

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest  = xgb.DMatrix(X_test, label=y_test)

# Define multi-class parameters
num_classes = len(train_df[target_col].unique())
params = {
    'objective': 'multi:softmax',  # or 'multi:softprob' for probabilities
    'num_class': num_classes,
    'eval_metric': 'mlogloss',
    'eta': 0.03,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

# Train
bst = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    evals=[(dtrain, 'train'), (dtest, 'test')],
    early_stopping_rounds=50,
    verbose_eval=50
)

# Predict
y_pred = bst.predict(dtest)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

[0]	train-mlogloss:1.01868	test-mlogloss:1.04682
[50]	train-mlogloss:0.68808	test-mlogloss:0.86252
[100]	train-mlogloss:0.56788	test-mlogloss:0.82095
[150]	train-mlogloss:0.49650	test-mlogloss:0.80358
[200]	train-mlogloss:0.44583	test-mlogloss:0.79214
[250]	train-mlogloss:0.40641	test-mlogloss:0.78553
[300]	train-mlogloss:0.37339	test-mlogloss:0.78076
[350]	train-mlogloss:0.34369	test-mlogloss:0.77704
[400]	train-mlogloss:0.31703	test-mlogloss:0.77495
[450]	train-mlogloss:0.29411	test-mlogloss:0.77343
[500]	train-mlogloss:0.27456	test-mlogloss:0.77373
[503]	train-mlogloss:0.27325	test-mlogloss:0.77350
Accuracy: 0.6492060594999087
              precision    recall  f1-score   support

           0       0.60      0.49      0.54       804
           1       0.66      0.43      0.52      2123
           2       0.65      0.89      0.75      2552

    accuracy                           0.65      5479
   macro avg       0.64      0.60      0.60      5479
weighted avg       0.65      0.65   

In [76]:
X_train = train_df[feature_cols]
y_train = train_df[target_col]

X_test  = test_df[feature_cols]
y_test  = test_df[target_col]

In [73]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score

cat_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03,
    depth=6,
    eval_metric='MultiClass',
    random_seed=42,
    early_stopping_rounds=50,
    verbose=50
)

cat_model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    use_best_model=True
)

# Predictions
y_pred = cat_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0:	learn: 1.0829594	test: 1.0863912	best: 1.0863912 (0)	total: 612ms	remaining: 10m 11s
50:	learn: 0.8049371	test: 0.8983701	best: 0.8983701 (50)	total: 16s	remaining: 4m 57s
100:	learn: 0.7360641	test: 0.8678690	best: 0.8678690 (100)	total: 31.5s	remaining: 4m 40s
150:	learn: 0.6986316	test: 0.8501974	best: 0.8501974 (150)	total: 46s	remaining: 4m 18s
200:	learn: 0.6732767	test: 0.8400038	best: 0.8400038 (200)	total: 1m 1s	remaining: 4m 3s
250:	learn: 0.6518248	test: 0.8312293	best: 0.8312293 (250)	total: 1m 16s	remaining: 3m 47s
300:	learn: 0.6349223	test: 0.8252971	best: 0.8252971 (300)	total: 1m 30s	remaining: 3m 30s
350:	learn: 0.6192274	test: 0.8196579	best: 0.8196579 (350)	total: 1m 45s	remaining: 3m 15s
400:	learn: 0.6051870	test: 0.8150366	best: 0.8150366 (400)	total: 2m	remaining: 2m 59s
450:	learn: 0.5921159	test: 0.8113962	best: 0.8113962 (450)	total: 2m 14s	remaining: 2m 43s
500:	learn: 0.5802143	test: 0.8081349	best: 0.8081349 (500)	total: 2m 28s	remaining: 2m 27s
550:	le

In [74]:
import lightgbm as lgb

In [79]:


# Create LightGBM dataset
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_test  = lgb.Dataset(X_test, label=y_test, reference=lgb_train)

# ----------------------
# Define parameters
# ----------------------
num_classes = len(np.unique(y_train))
params = {
    'objective': 'multiclass',
    'num_class': num_classes,
    'metric': 'multi_logloss',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0  # 0 means no output
}

# ----------------------
# Train with early stopping using callbacks
# ----------------------
lgb_model = lgb.train(
    params,
    lgb_train,
    num_boost_round=1000,
    valid_sets=[lgb_train, lgb_test],
    valid_names=['train','test'],
    callbacks=[lgb.early_stopping(stopping_rounds=50),
               lgb.log_evaluation(period=50)]  # print every 50 rounds
)

# ----------------------
# Predict
# ----------------------
y_pred = np.argmax(lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration), axis=1)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Training until validation scores don't improve for 50 rounds
[50]	train's multi_logloss: 0.579327	test's multi_logloss: 0.814891
[100]	train's multi_logloss: 0.471094	test's multi_logloss: 0.786878
[150]	train's multi_logloss: 0.39942	test's multi_logloss: 0.779509
[200]	train's multi_logloss: 0.344213	test's multi_logloss: 0.776606
Early stopping, best iteration is:
[195]	train's multi_logloss: 0.349079	test's multi_logloss: 0.775766
Accuracy: 0.6462858185800329
              precision    recall  f1-score   support

           0       0.59      0.49      0.54       804
           1       0.65      0.42      0.51      2123
           2       0.65      0.88      0.75      2552

    accuracy                           0.65      5479
   macro avg       0.63      0.60      0.60      5479
weighted avg       0.64      0.65      0.63   